In [15]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
!git clone "https://github.com/ScierKnave/honor_project.git"
import requests
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-a3ktyo8u
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-a3ktyo8u
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
fatal: destination path 'honor_project' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [16]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Student hyperparameters
# MPS parameters
bond_dim_HP = 40
# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 30 
student_lr_HP = 1e-4
student_reg_HP = 0
student_loss_HP = nn.CrossEntropyLoss()


# Premilinaries: Importing the data and utils subroutines

In [17]:

# Import the *common* mnist train set and create a batch iterator for it.
train_set = torch.load('/content/honor_project/src/datasets/train_mnist.pt')
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_train_HP)),
    batch_size=batch_sz_HP
    )

# Import the mnist *common* test set and create a batch iterator for it.
test_set = torch.load('/content/honor_project/src/datasets/test_mnist.pt')
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_test_HP)),
    batch_size = batch_sz_HP
    )

In [18]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator, dataset_size):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / dataset_size # divide by total size

# Training the student model

In [19]:
def train_a_student():
  '''
  Trains a student model.
  '''

  # Initialize the MPS module
  student = MPS(
      input_dim = 28 ** 2,
      output_dim = 10,
      bond_dim = bond_dim_HP
  ).to(chosen_device) 
  #student.register_feature_map(feature_map_HP)

  # Instantiate the optimizer and softmax
  student_optimizer = torch.optim.Adam(
      student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
  )

  # Used on the inputs before the loss function
  LogSoftmax = nn.LogSoftmax(dim=1)

  # Create an array to store the val loss
  # of the student at each epoch
  stud_test_loss = np.array([])
  stud_train_loss = np.array([])

  softmax = nn.Softmax(dim=1)

  stud_train_loss = np.array([])
  stud_test_loss = np.array([])

  for epoch in range(nepochs_student_HP):
      '''
      Training loop through the epochs.
      '''
      for (x_mb, y_mb) in train_iterator:
          # Flatten the MNIST images, which come in matrix form
          x_mb = x_mb.reshape(-1, 784).to(chosen_device)
          y_mb = y_mb.to(chosen_device)

          student_logits = student(x_mb) 

          # Backpropagation
          loss = student_loss_HP( student_logits, y_mb)

          student_optimizer.zero_grad()
          loss.backward()
          student_optimizer.step()

      # Get accuracy over all test and training data for current epoch
      train_current_accuracy = round( get_acc(student, train_iterator, nb_train_HP).item(), 4)
      test_current_accuracy = round( get_acc(student, test_iterator, nb_test_HP).item(), 4)
      stud_train_loss = np.append(stud_train_loss, train_current_accuracy)
      stud_test_loss = np.append(stud_test_loss, test_current_accuracy)
  return(stud_train_loss, stud_test_loss)


# Repeat the training process in order to get the variance
global_stud_test_loss = np.array([])
global_stud_train_loss = np.array([])
for i in range(20):
  print("\n Training and testing of model ", i+1, " in progress...")
  (stud_train_loss, stud_test_loss) = train_a_student()
  print("Train loss: ", stud_train_loss)
  print("Test loss: ", stud_test_loss)
  if (i == 0):
    global_stud_train_loss = stud_train_loss
    global_stud_test_loss = stud_test_loss
  else:
    global_stud_train_loss = np.vstack((global_stud_train_loss, stud_train_loss))
    global_stud_test_loss = np.vstack((global_stud_test_loss, stud_test_loss))




 Training and testing of model  1  in progress...
Train loss:  [0.1    0.1325 0.119  0.1892 0.4954 0.7119 0.7985 0.8355 0.8831 0.87
 0.9166 0.9297 0.9402 0.9599 0.9437 0.9718 0.98   0.9563 0.9568 0.9789
 0.9441 0.9768 0.982  0.9823 0.9797 0.9916 0.9882 0.9741 0.9761 0.9921]
Test loss:  [0.1    0.124  0.1118 0.1835 0.4762 0.711  0.7988 0.845  0.8723 0.87
 0.889  0.8876 0.8832 0.9021 0.8933 0.9009 0.9044 0.8982 0.8966 0.9173
 0.8726 0.9179 0.8834 0.9283 0.9154 0.9318 0.9168 0.8944 0.9181 0.925 ]

 Training and testing of model  2  in progress...
Train loss:  [0.143  0.197  0.5542 0.7564 0.8215 0.8062 0.8795 0.914  0.9235 0.9467
 0.9441 0.9545 0.9532 0.9517 0.9563 0.9619 0.971  0.9766 0.9824 0.9806
 0.9713 0.9776 0.9771 0.9889 0.9753 0.9881 0.9898 0.9901 0.9879 0.9807]
Test loss:  [0.1378 0.1981 0.5615 0.7873 0.8163 0.7939 0.8412 0.8878 0.8875 0.897
 0.896  0.9051 0.8796 0.9027 0.8967 0.9178 0.9225 0.9092 0.9162 0.9032
 0.8899 0.923  0.9068 0.9256 0.9115 0.9142 0.917  0.9345 0.9156 0.899

In [20]:
# Print the final results and save them for the report.
print("Final results")
print("Train loss: ", global_stud_train_loss)
print("Test loss: ", global_stud_test_loss)

np.save('20x_mps_trainloss_' + str(bond_dim_HP) + 'bd', global_stud_train_loss)
np.save('20x_mps_testloss_' + str(bond_dim_HP) + 'bd', global_stud_test_loss)


Final results
Train loss:  [[0.1    0.1325 0.119  0.1892 0.4954 0.7119 0.7985 0.8355 0.8831 0.87
  0.9166 0.9297 0.9402 0.9599 0.9437 0.9718 0.98   0.9563 0.9568 0.9789
  0.9441 0.9768 0.982  0.9823 0.9797 0.9916 0.9882 0.9741 0.9761 0.9921]
 [0.143  0.197  0.5542 0.7564 0.8215 0.8062 0.8795 0.914  0.9235 0.9467
  0.9441 0.9545 0.9532 0.9517 0.9563 0.9619 0.971  0.9766 0.9824 0.9806
  0.9713 0.9776 0.9771 0.9889 0.9753 0.9881 0.9898 0.9901 0.9879 0.9807]
 [0.1    0.1    0.1931 0.194  0.1776 0.521  0.714  0.7998 0.7881 0.8826
  0.9093 0.946  0.9442 0.9574 0.9574 0.9423 0.9663 0.9742 0.9546 0.9531
  0.9622 0.9786 0.9758 0.951  0.9848 0.9908 0.9828 0.974  0.9941 0.9966]
 [0.1    0.1    0.2144 0.3873 0.581  0.7764 0.8357 0.8883 0.907  0.9223
  0.9283 0.9216 0.9562 0.9613 0.9126 0.9712 0.9736 0.9778 0.9678 0.9571
  0.9724 0.9808 0.984  0.9847 0.9879 0.9897 0.9892 0.9728 0.9864 0.9878]
 [0.1222 0.1    0.3328 0.5704 0.7267 0.8096 0.8404 0.8748 0.9062 0.9008
  0.9273 0.9437 0.9536 0.9373 0.965